In [3]:
#yahooFinanceJapanから出来高率のデータを取得してCSVに出力してからMysqlのテーブルに登録する
#Pythonスクレイピング
#2021/02/18
#東証一部、東証二部ごとにわけて約３〜５ページをそれぞれ取得する。
#企業コードを格納するカラムはユニークにはしない。複数日で同じ企業コードが出現するから
#出来高率は倍の文字を抜いて数字として扱う
#連続して短時間にサーバにアクセスしないように、時間間隔を空けてアクセスする
#出来高率の取得は市場が終わって20分後がよい
#Mysql DB:YahooFinance, TABLE:Table_Dekidaka
######仕様######
#外部ファイルにクラス、メソッドを配置
#動作フロー
#ヤフーファイナンスのランキングから出来高率、東証一部、二部の数ページをスクレイピング
#↓
#CSVファイル出力、ファイルとして保存
#↓
#CSVファイル読み込み（同じファイル）
#↓
#Mysqlのテーブルにインサート
#↓
#PythonからSELECT文を使って条件を出力、もしかすると直接Mysqlを使ったほうが早いかも

######作業内容######
#ヤフーファイナンスのランキングから出来高率のページを開く
#今後、出来高以外の条件のデータを取得することもあるので汎用性を考慮する
#市場ごとに必要なページ数を取得する。
#ループが2個　URLの引数を調整
#市場を番号で管理する
#東証一部で約5ページを取得
#東証二部で約5ページを取得
#一旦CSVに出力する（テスト、動作確認のため）
#CSVを読み込みMYｓｑｌの所定のテーブルに追加する
#SELECT文で必要な情報を出力する、TEXTかCSV

#ADD　2021/02/18　
#yahooFinanceJapanから出来高率のデータを取得してMysqlのテーブルに登録する
#URL　https://info.finance.yahoo.co.jp/ranking/
#kd= ランキング種別項目koumoku 、mk= 市場market
#kd=33 出来高、kd=50 一株当期利益EPS 
#mk=3 東証一部、 mk=4東証二部
#株式ランキング、出来高率、東証一部
#https://info.finance.yahoo.co.jp/ranking/?kd=33&mk=3&tm=d&vl=a
#https://info.finance.yahoo.co.jp/ranking/?kd=33&tm=d&vl=a&mk=3&p=1
#https://info.finance.yahoo.co.jp/ranking/?kd=33&tm=d&vl=a&mk=3&p=2

#株式ランキング、出来高率、東証二部
#https://info.finance.yahoo.co.jp/ranking/?kd=33&mk=4&tm=d&vl=a

#一株当期利益　東証一部
#https://info.finance.yahoo.co.jp/ranking/?kd=50&mk=3&tm=d&vl=a
#一株当期利益東証二部
#https://info.finance.yahoo.co.jp/ranking/?kd=50&mk=4&tm=d&vl=a

import MySQLdb
import os
import re
import sys
import time
import datetime
import glob#ファイル一覧
import shutil#move
from selenium import webdriver
import importlib#再読み込みのためのライブラリ

import methodMysql#個人用メソッド pyファイル

###START
importlib.reload(methodMysql)

print("START#####################################")
dekidakaObj=methodMysql.yahooFinanceDekidaka()
#kd=33 出来高,mk=3 東証一部 , pageNum
dekidakaNum=33
getArray=[]#各種データのカプセルを入れた配列、クラスを使用
marketArray=[3,4]#東証一部、二部、3,4
for f in marketArray:
    for i in range(1,6):#iは1,2になる3は含まない。つまり i<3のループとなる
        #配列は結合＋できる。a=[1,2]、b[3,4]はa+bで[1,2,3,4]とできるのでgetArrayに結合
        getArray+=dekidakaObj.getDekidakaTest(dekidakaNum,f,i)#33出来高koumokuCode,marketCode,pageNum
        
#CSV出力（動作確認のためにCSV出力する、コードにバグがあったときに不具合を修正作業しやすいから）
#出来高率20120219.csv　そしてそのファイルが有った場合はすでに取り込み済みとしたい。
#もう一度取り込みたいときは、テーブルを削除して、CSVも消す作業が必要
filename='出来高率.csv'
#出力するファイル名＞＞出来高率20210219.csvとして出力される
#日付を取得して、後ろに付ける
dt_now=datetime.datetime.now()
nameDate=dt_now.strftime('%Y%m%d')
filename=filename.split('.')[0]+str(nameDate)+'.csv'#＞＞出来高率20210219.csvとして出力される
path='/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/'

#ファイルチェック、すでにファイルが存在した場合は、作業を中止する。すでに取り込み済みと判断する。
#平日に複数回実行しようとするとエラーになる。実行日の日付のファイル名をチェックするから。
#この場合、土日とか祝日に実行すると日付が異なるので再取り込みできてしまうが、実行者がそのことに気づいているという前提で
#事前に取り込んだかをTableを調べるか、すでに作成されているCSVファイルの日付から推測できると思う。
#つまり金曜日にすでに取り込んだが、うっかりして土曜日に再度実行すると土曜日は市場は休みであるため同じデータがヤフーファイナンスで
#表示されていて取り込めてしまう。取り込み実行をするときは、念の為にCSVファイルを確認した上で取り込んだかを知っておくようにする。
#もし2重にインサートした場合は、更新日付で判断して直接テーブルデータを手動（クエリを作成）で削除する。
#仮に2重で登録されても、同じデータが表示されるだけである。集計に影響はない。
#使用においては特に株価を合計するわけでもない。出来高の上昇具合を把握するためである。

if dekidakaObj.checkFileExist(filename,path)==True:
    print("すでにファイルが存在しますので、作業を中止。テーブルデータを削除してCSVファイルを削除してください。")
    print("DB:YahooFinance, TABLE:Table_Dekidaka ,user:maseda ,pass:Tes****")#passは知られてもOKなパスワード
    sys.exit()

if len(getArray)>0:#1個以上取得で実行、空の場合は何もしない
    dekidakaObj.outputCSVForTableColum(getArray,filename,path)

    if dekidakaObj.checkFileExist(filename,path)==True:#CSV出力されたか確認する。CSVがないと作業しない。
        #CSVをMysqlへ
        #インサート実行、2021/02/21
        dekidakaObj.CSVtoMysqlConnector(filename,path)#MysqlのテーブルにCSVデータを登録する。DBとテーブルは事前に作成済
        #mysql select 表示
        dekidakaObj.selectDekidakaMysql()
    else:
        print("CSVファイルが出力されていません。チェックしてください")

print("End#####################################")
# エラー対応
#mysql サーバーエラーの場合、サーバーが起動していない事がある。ターミナルから以下のコマンドを実行する
#　mysql.server start

#SQL 実行例、出来高1万以上、株価1000円以上　倍率3倍以上　
#mysql> select Code,Market,Name,Price,Volume,PreviousVolume from Table_Dekidaka
#where StockDate='2021-02-22' and Volume>='100000' and VolumePer>=3 and Price>=1000;

START#####################################











/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/test.csv
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/出来高率20210221.csv
ファイルはありません
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/test.csv
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/出来高率20210222.csv
ファイルは存在します
Trueなら接続OK
True
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data
commit
DB 処理終了。。。
End#####################################


In [48]:
import MySQLdb
import os
import re
import sys
import datetime
import glob#ファイル一覧
import shutil#move
from selenium import webdriver
import importlib#再読み込みのためのライブラリ

import methodMysql#個人用メソッド pyファイル

###START
importlib.reload(methodMysql)
filename='出来高率.csv'#出力するファイル名
path='/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/'
obj=methodMysql.yahooFinanceDekidaka()

obj.checkFileExist(filename,path)
#obj.CSVtoMysqlConnectorTest(filename,path)

dt_now=datetime.datetime.now()
nameDate=dt_now.strftime('%Y%m%d')
print(nameDate)


/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/test.csv
ファイルはありません
20210221


In [15]:
#今度はEPS、1株あたりの利益のデータを取得
#注意事項　2021/02/23
#kessanDateは年月までが正しくて、決算月である。テーブルデータはDATE型なので、日を入れているが無作為に1日になっているだけ。
#ヤフーファイナンスの1株利益のデータは「決算月」なので年月までしかない。

#yahooFinanceJapanからEPS,1株あたりの利益のデータを取得してCSVに出力してからMysqlのテーブルに登録する
#Pythonスクレイピング
#作業内容は、出来高率と同じ
#2021/02/23
#東証一部、東証二部ごとにわけて約３〜５ページをそれぞれ取得する。
#企業コードを格納するカラムはユニークにはしない。複数日で同じ企業コードが出現するから
#出来高率は倍の文字を抜いて数字として扱う
#連続して短時間にサーバにアクセスしないように、時間間隔を空けてアクセスする
#出来高率の取得は市場が終わって20分後がよい
#Mysql DB:YahooFinance, TABLE:Table_EPS
######仕様######
#外部ファイルにクラス、メソッドを配置
#動作フロー
#ヤフーファイナンスのランキングからEPS、東証一部、二部の数ページをスクレイピング
#↓
#CSVファイル出力、ファイルとして保存
#↓
#CSVファイル読み込み（同じファイル）
#↓
#Mysqlのテーブルにインサート
#↓
#PythonからSELECT文を使って条件を出力、もしかすると直接Mysqlを使ったほうが早いかも

######作業内容######
#ヤフーファイナンスのランキングから出来高率のページを開く
#今後、出来高以外の条件のデータを取得することもあるので汎用性を考慮する
#市場ごとに必要なページ数を取得する。
#ループが2個　URLの引数を調整
#市場を番号で管理する
#東証一部で約5ページを取得
#東証二部で約5ページを取得
#一旦CSVに出力する（テスト、動作確認のため）
#CSVを読み込みMYｓｑｌの所定のテーブルに追加する
#SELECT文で必要な情報を出力する、TEXTかCSV

#ADD　2021/02/18　
#yahooFinanceJapanから出来高率のデータを取得してMysqlのテーブルに登録する
#URL　https://info.finance.yahoo.co.jp/ranking/
#kd= ランキング種別項目koumoku 、mk= 市場market
#kd=33 出来高、kd=50 一株当期利益EPS 
#mk=3 東証一部、 mk=4東証二部
#株式ランキング、出来高率、東証一部
#https://info.finance.yahoo.co.jp/ranking/?kd=33&mk=3&tm=d&vl=a
#https://info.finance.yahoo.co.jp/ranking/?kd=33&tm=d&vl=a&mk=3&p=1
#https://info.finance.yahoo.co.jp/ranking/?kd=33&tm=d&vl=a&mk=3&p=2

#株式ランキング、出来高率、東証二部
#https://info.finance.yahoo.co.jp/ranking/?kd=33&mk=4&tm=d&vl=a

#一株当期利益　東証一部
#https://info.finance.yahoo.co.jp/ranking/?kd=50&mk=3&tm=d&vl=a
#一株当期利益東証二部
#https://info.finance.yahoo.co.jp/ranking/?kd=50&mk=4&tm=d&vl=a

import MySQLdb
import os
import re
import sys
import time
import datetime
import glob#ファイル一覧
import shutil#move
from selenium import webdriver
import importlib#再読み込みのためのライブラリ

import methodMysql#個人用メソッド pyファイル

###START
importlib.reload(methodMysql)
print("１株利益　EPS#####################################")

print("START#####################################")
epsObj=methodMysql.yahooFinanceEps()
#kd=50 EPS,mk=3 東証一部 , pageNum
dekidakaNum=50#　　EPSは50
getArray=[]#各種データのカプセルを入れた配列、クラスを使用
marketArray=[3,4]#東証一部、二部、3,4
for f in marketArray:
    for i in range(1,8):#iは1,2になる3は含まない。つまり i<3のループとなる
        #配列は結合＋できる。a=[1,2]、b[3,4]はa+bで[1,2,3,4]とできるのでgetArrayに結合
        getArray+=epsObj.getEps(dekidakaNum,f,i)#50Eps koumokuCode,marketCode,pageNum
        
#CSV出力（動作確認のためにCSV出力する、コードにバグがあったときに不具合を修正作業しやすいから）
#eps20120219.csv　そしてそのファイルが有った場合はすでに取り込み済みとしたい。
#もう一度取り込みたいときは、テーブルを削除して、CSVも消す作業が必要
filename='eps.csv'
#出力するファイル名＞＞出来高率20210219.csvとして出力される
#日付を取得して、後ろに付ける
dt_now=datetime.datetime.now()
nameDate=dt_now.strftime('%Y%m%d')
filename=filename.split('.')[0]+str(nameDate)+'.csv'#＞＞eps20210219.csvとして出力される
path='/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/'

#ファイルチェック、すでにファイルが存在した場合は、作業を中止する。すでに取り込み済みと判断する。
#平日に複数回実行しようとするとエラーになる。実行日の日付のファイル名をチェックするから。
#この場合、土日とか祝日に実行すると日付が異なるので再取り込みできてしまうが、実行者がそのことに気づいているという前提で
#事前に取り込んだかをTableを調べるか、すでに作成されているCSVファイルの日付から推測できると思う。
#つまり金曜日にすでに取り込んだが、うっかりして土曜日に再度実行すると土曜日は市場は休みであるため同じデータがヤフーファイナンスで
#表示されていて取り込めてしまう。取り込み実行をするときは、念の為にCSVファイルを確認した上で取り込んだかを知っておくようにする。
#もし2重にインサートした場合は、更新日付で判断して直接テーブルデータを手動（クエリを作成）で削除する。
#仮に2重で登録されても、同じデータが表示されるだけである。集計に影響はない。
#使用においては特に株価を合計するわけでもない。出来高の上昇具合を把握するためである。

if epsObj.checkFileExist(filename,path)==True:
    print("すでにファイルが存在しますので、作業を中止。テーブルデータを削除してCSVファイルを削除してください。")
    print("DB:YahooFinance, TABLE:Table_Eps ,user:maseda ,pass:Tes****")#passは知られてもOKなパスワード
    sys.exit()

if len(getArray)>0:#1個以上取得で実行、空の場合は何もしない
    epsObj.outputCSVForTableColum(getArray,filename,path)

    if epsObj.checkFileExist(filename,path)==True:#CSV出力されたか確認する。CSVがないと作業しない。
        #CSVをMysqlへ
        #インサート実行、2021/02/21
        epsObj.CSVtoMysqlConnector(filename,path)#MysqlのテーブルにCSVデータを登録する。DBとテーブルは事前に作成済
        #mysql select 表示
        epsObj.selectDekidakaMysql()
    else:
        print("CSVファイルが出力されていません。チェックしてください")

print("EPS　1株利益#####################################")
print("End#####################################")

# エラー対応
#mysql サーバーエラーの場合、サーバーが起動していない事がある。ターミナルから以下のコマンドを実行する
#　mysql.server start


#出来高の場合
#SQL 実行例、株価1000円以上4500円以下　EPS300円以上
#mysql> select code,market,name,price,eps,kessanDate from Table_eps where price>=1000 and price<=4500 
#and StockDate='2021/02/23' and eps>=200 and market='東証1部' order by eps desc;

#mysql> select code,market,name,price,eps,kessanDate from Table_eps where price>=1000 and price<=4500 
#and StockDate='2021/02/23' and eps>=300 and market='東証2部' order by eps desc;

１株利益　EPS#####################################
START#####################################





































































































/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/test.csv
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/出来高率20210222.csv
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/出来高率20210221.csv
ファイルはありません
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data/eps20210223.csv
ファイルは存在します
Trueなら接続OK
True
/Users/toshiromaseda/Documents/2021年/2021年株/yahoofinance_data
commit
DB 処理終了。。。
    ID  Code Market            Name   Price KessanSyubetsu   Eps  KessanDate  \
0  101  7974   東証1部          任天堂(株)   68030            (連)  3358  2021-03-01   
1  102  9983   東証1部  (株)ファーストリテイリング  104900            (連)  1616  2021-08-01   
2  103  6273   東証1部          ＳＭＣ(株)   65860            (連)  1579  2021-03-01   
3  